In [ ]:
#pip install elasticsearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 385 kB 6.0 MB/s 
     |████████████████████████████████| 59 kB 4.2 MB/s 
     |████████████████████████████████| 140 kB 49.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.13 which is incompatible.


# Módulos

In [44]:
# Importacion de modulos 
import elasticsearch
import numpy as np
from elasticsearch import Elasticsearch
from elasticsearch import client
from datetime import datetime
from time import sleep

# Inputs 

In [45]:
# Informacion sobre la incidencia
fecha_inicio_incidencia = "2022-03-04 23:50:00" # Meter la hora según timezone
fecha_fin_incidencia    = "2022-03-06 00:10:00" # Meter la hora según timezone

# Nombre del Job de Elastic
jobname                 = "sabado_anomalia"

In [46]:
# Datos de la conexion con el servidor de Elastic
ELASTIC_PASSWORD = "9QJVpAsI1dfU33vZpc072VTd"
CLOUD_ID = "SeriestemporalesElastic:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQxMmRiODkwZjhhN2U0YTVjOTAwMDViODMyZjhlYWViNCRjMmU4YmQxYjkzZTI0MGY5OTZiYzIyNTFkZGMxMGY4Yg=="

client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)

# Nuevas Variables

In [47]:
# Calculo del offset por cambio de hora

# Timestamps fechas incidencia
fecha_inicio_incidencia = datetime.strptime(fecha_inicio_incidencia, '%Y-%m-%d %H:%M:%S')
fecha_inicio_incidencia_timestamp = datetime.timestamp(fecha_inicio_incidencia)*1000

fecha_fin_incidencia = datetime.strptime(fecha_fin_incidencia, '%Y-%m-%d %H:%M:%S')
fecha_fin_incidencia_timestamp = datetime.timestamp(fecha_fin_incidencia)*1000

# Timestamps de las fechas de cambio de hora

# Primer cambio de hora
fecha_cambio_hora_1 = "2022-03-27 02:00:00" 
fecha_cambio_hora_1 = datetime.strptime(fecha_cambio_hora_1, '%Y-%m-%d %H:%M:%S')
fecha_cambio_hora_1_timestamp = datetime.timestamp(fecha_cambio_hora_1)*1000

# Segundo cambio de hora
fecha_cambio_hora_2 = "2022-10-30 02:00:00" 
fecha_cambio_hora_2 = datetime.strptime(fecha_cambio_hora_2, '%Y-%m-%d %H:%M:%S')
fecha_cambio_hora_2_timestamp = datetime.timestamp(fecha_cambio_hora_2)*1000

# Comparamos fechas en el mismo año 
fecha_inicio_incidencia_cambio_timestamp = (datetime(2022, fecha_inicio_incidencia.month, fecha_inicio_incidencia.day, fecha_inicio_incidencia.hour, fecha_inicio_incidencia.minute, fecha_inicio_incidencia.second).timestamp())*1000 

if (fecha_inicio_incidencia_cambio_timestamp < fecha_cambio_hora_1_timestamp):
    offset_1 = -3600 

elif ((fecha_inicio_incidencia_cambio_timestamp > fecha_cambio_hora_1_timestamp) and (fecha_inicio_incidencia_cambio_timestamp < fecha_cambio_hora_2_timestamp)):
  offset_1 = -7200

else (fecha_cambio_hora_2_timestamp < fecha_inicio_incidencia_cambio_timestamp):
  offset_1 = -3600  

fecha_fin_incidencia_cambio_timestamp = (datetime(2022, fecha_fin_incidencia.month, fecha_fin_incidencia.day, fecha_fin_incidencia.hour, fecha_fin_incidencia.minute, fecha_fin_incidencia.second).timestamp())*1000 

if (fecha_fin_incidencia_cambio_timestamp < fecha_cambio_hora_1_timestamp):
    offset_2 = -3600 

elif ((fecha_fin_incidencia_cambio_timestamp > fecha_cambio_hora_1_timestamp) and (fecha_fin_incidencia_cambio_timestamp < fecha_cambio_hora_2_timestamp)):
  offset_2 = -7200

else (fecha_cambio_hora_2_timestamp < fecha_fin_incidencia_cambio_timestamp):
  offset_2 = -3600  

In [48]:
# Transformacion de variables de entrada
# Fechas
fecha_inicio_incidencia = str((int(fecha_inicio_incidencia_timestamp + offset_1*1000)))
fecha_fin_incidencia = str((int(fecha_fin_incidencia_timestamp + offset_2*1000)))

# Nombre del Datafeed
datafeed_id = "datafeed-" + jobname

# Cliente de ML
cliente_ml = client.ml

# Solución

In [138]:
# Paso 1. Parar y cerrar el Job
cliente_ml.close_job(job_id=jobname)

ObjectApiResponse({'closed': True})

In [139]:
# Paso 2. Escoger el id de la snapshot con latest_record_time_stamp mas cercano a la incidencia
snapshots_available = cliente_ml.get_model_snapshots(job_id=jobname)
# Recoger el count 
number_snapshots = snapshots_available["count"]
# Iterar guardando en una matriz el id del snpashot y latest_record_time_stamp
snapshots_info = snapshots_available["model_snapshots"]
latest = np.zeros((number_snapshots, 2))
for i in range(number_snapshots):
  latest[i,0] = snapshots_info[i]["latest_record_time_stamp"]

for i in range(number_snapshots):
  latest[i,1] = snapshots_info[i]["snapshot_id"]

# Comparar con la fecha de la incidencia para buscar cual esta inmediatamente antes
  # Ordenar por timestamp (primera columna)
  latest1 = latest[latest[:,0].argsort()]
  latest2 = np.flip(latest1, axis=0)
  # Eliminar valores mayores
  latest3 = latest2[latest2[:,0]<float(fecha_inicio_incidencia)]
# Escoger el id de la reversion correcta
id_revert = latest3[0,1]
id_revert = str(int(id_revert))

TypeError: ignored

In [ ]:
# Paso 3. Revertir el Job antes de la incidencia 
cliente_ml.revert_model_snapshot(job_id=jobname, snapshot_id=id_revert, delete_intervening_results=True)
sleep(5)

In [ ]:
# Paso 4. Abrir el Job 
cliente_ml.open_job(job_id=jobname)
sleep(5)

In [ ]:
# Paso 5. Avanzamos datafeed hasta la fecha de inicio de incidencia
cliente_ml.start_datafeed(datafeed_id=datafeed_id, end=fecha_inicio_incidencia)
state_datafeed = 1

# Una vez completado el paso continuamos al siguiente
while (state_datafeed == 1):
  sleep(10)
  state_datafd = cliente_ml.get_datafeed_stats(datafeed_id=datafeed_id)
  st = state_datafd['datafeeds']
  state_actual = st[0]['state'] 
  if (state_actual == 'stopped'):
    state_datafeed = 0

In [ ]:
# Paso 6. Abrir el Job 
cliente_ml.open_job(job_id=jobname)
sleep(5)

In [ ]:
# Paso 7. Avanzamos datafeed desde la fecha de fin de incidencia hasta la fecha actual
cliente_ml.start_datafeed(datafeed_id=datafeed_id, start=fecha_fin_incidencia)

# Parte de Indices

In [400]:
#!pip install eland
#!pip install elasticsearch

In [401]:
# Importacion de modulos 
import elasticsearch
import numpy as np
from elasticsearch import Elasticsearch
from elasticsearch import client
from datetime import datetime
from time import sleep
from elasticsearch.client import IndicesClient
import eland as ed
import pandas as pd
from elasticsearch.client import IngestClient

# Inputs

In [402]:
# Informacion sobre la incidencia
fecha_inicio_incidencia = "2022-03-04 23:50:00" # Meter la hora segun timezone, no en UTC
fecha_fin_incidencia    = "2022-03-06 00:10:00" # Meter la hora según timezone, no en UTC

# Nombre del Job de Elastic
jobname                 = "sabado_anomalia"
variable_name_predict   = "y_an"

# Nombre del indice sobre el que se realiza el Job
index_data              = 'sabado_anomalia'

# Nuevas variables

In [403]:
# Calculo del offset por cambio de hora

# Timestamps fechas incidencia
fecha_inicio_incidencia = datetime.strptime(fecha_inicio_incidencia, '%Y-%m-%d %H:%M:%S')
fecha_inicio_incidencia_timestamp = datetime.timestamp(fecha_inicio_incidencia)*1000

fecha_fin_incidencia = datetime.strptime(fecha_fin_incidencia, '%Y-%m-%d %H:%M:%S')
fecha_fin_incidencia_timestamp = datetime.timestamp(fecha_fin_incidencia)*1000

# Timestamps de las fechas de cambio de hora

# Primer cambio de hora
fecha_cambio_hora_1 = "2022-03-27 02:00:00" 
fecha_cambio_hora_1 = datetime.strptime(fecha_cambio_hora_1, '%Y-%m-%d %H:%M:%S')
fecha_cambio_hora_1_timestamp = datetime.timestamp(fecha_cambio_hora_1)*1000

# Segundo cambio de hora
fecha_cambio_hora_2 = "2022-10-30 02:00:00" 
fecha_cambio_hora_2 = datetime.strptime(fecha_cambio_hora_2, '%Y-%m-%d %H:%M:%S')
fecha_cambio_hora_2_timestamp = datetime.timestamp(fecha_cambio_hora_2)*1000

# Comparamos fechas en el mismo año 
fecha_inicio_incidencia_cambio_timestamp = (datetime(2022, fecha_inicio_incidencia.month, fecha_inicio_incidencia.day, fecha_inicio_incidencia.hour, fecha_inicio_incidencia.minute, fecha_inicio_incidencia.second).timestamp())*1000 

if (fecha_inicio_incidencia_cambio_timestamp < fecha_cambio_hora_1_timestamp):
    offset_1 = -3600 

if ((fecha_inicio_incidencia_cambio_timestamp > fecha_cambio_hora_1_timestamp) and (fecha_inicio_incidencia_cambio_timestamp < fecha_cambio_hora_2_timestamp)):
  offset_1 = -7200

if (fecha_cambio_hora_2_timestamp < fecha_inicio_incidencia_cambio_timestamp):
  offset_1 = -3600  

fecha_fin_incidencia_cambio_timestamp = (datetime(2022, fecha_fin_incidencia.month, fecha_fin_incidencia.day, fecha_fin_incidencia.hour, fecha_fin_incidencia.minute, fecha_fin_incidencia.second).timestamp())*1000 

if (fecha_fin_incidencia_cambio_timestamp < fecha_cambio_hora_1_timestamp):
    offset_2 = -3600 

if ((fecha_fin_incidencia_cambio_timestamp > fecha_cambio_hora_1_timestamp) and (fecha_fin_incidencia_cambio_timestamp < fecha_cambio_hora_2_timestamp)):
  offset_2 = -7200

if (fecha_cambio_hora_2_timestamp < fecha_fin_incidencia_cambio_timestamp):
  offset_2 = -3600  

In [404]:
# Transformacion de variables de entrada
# Fechas
fecha_inicio_incidencia = (fecha_inicio_incidencia_timestamp + offset_1*1000)/1000
fecha_inicio_incidencia_snapshot = fecha_inicio_incidencia*1000
fecha_fin_incidencia    = (fecha_fin_incidencia_timestamp + offset_2*1000)/1000
fecha_fin_incidencia_snapshot = fecha_fin_incidencia*1000
# Avanzamos esta ultima fecha 2 hora 
fecha_fin_incidencia_snapshot_avanc = str((fecha_fin_incidencia+7200)*1000)

fecha_inicio_incidencia_datetime = datetime.fromtimestamp(fecha_inicio_incidencia)
fecha_fin_incidencia_datetime = datetime.fromtimestamp(fecha_fin_incidencia)

fecha_inicio_incidencia_datetime_str = str(fecha_inicio_incidencia_datetime)
fecha_fin_incidencia_datetime_str = str(fecha_fin_incidencia_datetime)
fecha_inicio_incidencia_datetime_querie = fecha_inicio_incidencia_datetime_str[0:10] + 'T' + fecha_inicio_incidencia_datetime_str[11:]
fecha_fin_incidencia_datetime_querie    = fecha_fin_incidencia_datetime_str[0:10] + 'T' + fecha_fin_incidencia_datetime_str[11:] 

# Nombre del Datafeed
datafeed_id = "datafeed-" + jobname

In [405]:
# Datos de la conexion con el servidor de Elastic
ELASTIC_PASSWORD = "9QJVpAsI1dfU33vZpc072VTd"
CLOUD_ID = "SeriestemporalesElastic:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQxMmRiODkwZjhhN2U0YTVjOTAwMDViODMyZjhlYWViNCRjMmU4YmQxYjkzZTI0MGY5OTZiYzIyNTFkZGMxMGY4Yg=="

es_client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD)
)
cliente_index = IndicesClient(client=es_client)

# Solución

In [406]:
# Creacion del Indice de Backup
indexname = 'indicebackup'
indexname_2 = 'indicebackup2'
indexname_3 = "indicebackup3"

try:
  cliente_index.create(index=indexname)
except:
  cliente_index.delete(index=indexname)
  cliente_index.create(index=indexname)

try:
  cliente_index.create(index=indexname_2)
except:
  cliente_index.delete(index=indexname_2)
  cliente_index.create(index=indexname_2)

try:
  cliente_index.create(index=indexname_3)
except:
  cliente_index.delete(index=indexname_3)
  cliente_index.create(index=indexname_3)

In [407]:
# Reindexar datos de la anomalia al indice backup 
es_client.reindex(body={
      "source": {
          "index": index_data,
          "query": {
              "range":{
                  "@timestamp":{
                      "gte": fecha_inicio_incidencia_datetime_querie,
                      "lte": fecha_fin_incidencia_datetime_querie
                      }
                   }
                }
              },
              "dest": {
                  "index": indexname
              }})

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  from ipykernel import kernelapp as app


ObjectApiResponse({'took': 40, 'timed_out': False, 'total': 147, 'updated': 0, 'created': 147, 'deleted': 0, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []})

In [408]:
# Obtener datos de la prediccion del Job para el tiempo de anomalia
df = ed.DataFrame(es_client, es_index_pattern=".ml-anomalies-shared")
# Filtramos para nuestro job de interes
df = df[df["job_id"] == jobname]
# Obtenemos las columnas de interes del dataframe
df_select = df[["timestamp", "model_median", "model_upper"]]
# Eliminar filas con valores nulos
df_select = ed.eland_to_pandas(df_select)
df_select = df_select.dropna()
df_select.head(20)
# Ordenamos filas por timestamp
df_select.timestamp
# Seleccionamos timestamp de interes
df_select['timestamp'] = pd.to_datetime(df_select.timestamp)
df_select_order = df_select.sort_values(by="timestamp", ascending=True)
df_select_order = df_select_order.set_index("timestamp")
df_select_filter = df_select_order.loc[fecha_inicio_incidencia_datetime:fecha_fin_incidencia_datetime]

In [409]:
# Hacer un rename del nombre de columna "model_median" al nombre de la variable que se predice en el JOB
df_select_filter.rename(columns = {'model_median':variable_name_predict}, inplace = True)

# Eliminar variable upper_name
df_select_filter = df_select_filter.drop('model_upper', axis=1)
df_select_filter = df_select_filter.reset_index()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [410]:
# Crear la pipeline de ingesta para @timestamp 
client_Ingest = IngestClient(client=es_client)

try:
  client_Ingest.put_pipeline(id='procesador_date_anomalias', body={
      'description': "Parsea el @timestamp",
      'processors': [
          {
              "date": {
                  "field": "timestamp", 
                  "formats": ["ISO8601"],
                  "target_field": "@timestamp"
                  }
          }
          ]})
except:
  pass

# Introducir datos a un indice intermedio
df_select_filter
for i, row in df_select_filter.iterrows():
    doc = {
        "timestamp": row["timestamp"],
        "y_an": row["y_an"]
    }
    es_client.index(index=indexname_2, id=i, document=doc)

# Eliminar del indice principal datos para el timestamp de la incidencia
es_client.delete_by_query(index=index_data, body={
  "query": {
    "range":{
         "@timestamp":{
            "gte": fecha_inicio_incidencia_datetime_querie,
            "lte": fecha_fin_incidencia_datetime_querie
          }
        }
  }
})
sleep(5)
# Parsear a un indice los datos teniendo cuidado con el timestamp al indice principal
es_client.reindex(body={
      'source': {
          "index": indexname_2
          },
          'dest': {
              "index": index_data,
              "pipeline": "procesador_date_anomalias"
              }},
              wait_for_completion=True)
sleep(5)
# Parsear a un indice los datos teniendo cuidado con el timestamp al indice principal
es_client.reindex(body={
      'source': {
          "index": indexname_2
          },
          'dest': {
              "index": index_data,
              "pipeline": "procesador_date_anomalias"
              }},
              wait_for_completion=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:60: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.


ObjectApiResponse({'took': 15, 'timed_out': False, 'total': 147, 'updated': 147, 'created': 0, 'deleted': 0, 'batches': 1, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []})

In [411]:
# Resetear Job desde la snapshot (buscar snapshot como en el modelo anterior y correr job)
# Paso 1. Parar y cerrar el Job
cliente_ml.close_job(job_id=jobname)
# Paso 2. Escoger el id de la snapshot con latest_record_time_stamp mas cercano a la incidencia
snapshots_available = cliente_ml.get_model_snapshots(job_id=jobname)
# Recoger el count 
number_snapshots = snapshots_available["count"]
# Iterar guardando en una matriz el id del snpashot y latest_record_time_stamp
snapshots_info = snapshots_available["model_snapshots"]
latest = np.zeros((number_snapshots, 2))
for i in range(number_snapshots):
  latest[i,0] = snapshots_info[i]["latest_record_time_stamp"]

for i in range(number_snapshots):
  latest[i,1] = snapshots_info[i]["snapshot_id"]

# Comparar con la fecha de la incidencia para buscar cual esta inmediatamente antes
  # Ordenar por timestamp (primera columna)
  latest1 = latest[latest[:,0].argsort()]
  latest2 = np.flip(latest1, axis=0)
  # Eliminar valores mayores
  latest3 = latest2[latest2[:,0]<float(fecha_inicio_incidencia_snapshot)]
# Escoger el id de la reversion correcta
id_revert = latest3[0,1]
id_revert = str(int(id_revert))
# Paso 3. Revertir el Job antes de la incidencia 
cliente_ml.revert_model_snapshot(job_id=jobname, snapshot_id=id_revert, delete_intervening_results=True)
# Paso 4. Abrir el Job 
cliente_ml.open_job(job_id=jobname)
# Paso 5. Avanzamos datafeed hasta la fecha de inicio de incidencia
cliente_ml.start_datafeed(datafeed_id=datafeed_id, end=str(fecha_fin_incidencia_snapshot))
state_datafeed = 1

# Una vez completado el paso continuamos al siguiente
while (state_datafeed == 1):
  sleep(10)
  state_datafd = cliente_ml.get_datafeed_stats(datafeed_id=datafeed_id)
  st = state_datafd['datafeeds']
  state_actual = st[0]['state'] 
  if (state_actual == 'stopped'):
    state_datafeed = 0

# Borrado de Indices

In [412]:
# Sustituir datos del indice backup al indice original para el tiempo de incidencia
# 1. Eliminar datos ingestados de la prediccion para el tiempo de incidencia
es_client.delete_by_query(index=index_data, body={
  "query": {
    "range":{
         "@timestamp":{
            "gte": fecha_inicio_incidencia_datetime_querie,
            "lte": fecha_fin_incidencia_datetime_querie
          }
        }
  }
})
sleep(5)
# 2. Ingesta de los datos del backup para el tiempo de incidencia
es_client.reindex(body={
      "source": {
          "index": indexname,
          "query": {
              "range":{
                  "@timestamp":{
                      "gte": fecha_inicio_incidencia_datetime_querie,
                      "lte": fecha_fin_incidencia_datetime_querie
                      }
                   }
                }
              },
              "dest": {
                  "index": index_data
              }},
              wait_for_completion=True)
sleep(5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.


In [413]:
# Eliminar indices creados
try:
  cliente_index.delete(index=indexname)
except:
  pass
try:
  cliente_index.delete(index=indexname_2)
except:
  pass
try:
  cliente_index.delete(index=indexname_3)
except:
  pass

In [414]:
# Paso 6. Avanzamos datafeed hasta el tiempo real
cliente_ml.open_job(job_id=jobname)
sleep(5)
cliente_ml.start_datafeed(datafeed_id=datafeed_id, start=str(fecha_fin_incidencia_snapshot))

ObjectApiResponse({'started': True, 'node': 'CxrRZr0BSsmLbYiv4E042Q'})